# Setup

In [1]:
!pip --quiet install ipython-autotime
%load_ext autotime

# set path for libraries
import sys
sys.path.append('/content/pytorch3d_rendering')

# --- Config ---
mount_gdrive = False

# --- Imports (module you saved as sse_env.py) ---
import importlib, os, sys, shutil, subprocess, urllib.request, pathlib
import installation_tools as install_tools
importlib.reload(install_tools)

# --- Short helpers (no notebook magics) ---
def run(cmd, check=True):
    print("$", " ".join(cmd))
    try:
        subprocess.run(cmd, check=check)
    except subprocess.CalledProcessError as e:
        print(f"Command failed ({e.returncode}): {' '.join(cmd)}")
        if check:
            raise

def pip_install(*pkgs, extra=None, check=True):
    args = [sys.executable, "-m", "pip", "install"]
    if extra:
        args += extra
    args += list(pkgs)
    run(args, check=check)

def conda_available():
    return shutil.which("conda") is not None

def conda_install(*pkgs):
    if not conda_available():
        print("conda not available; skipping conda installs.")
        return
    # Use -c conda-forge channel and auto-yes
    run(["conda", "install", "-y", "-c", "conda-forge", *pkgs], check=False)

# --- Detect platform ---
pm = install_tools.PlatformManager()
platform, local_path = pm.platform, pm.local_path
print("Detected:", platform, local_path)

# --- Optional: Mount GDrive if on Colab ---
if mount_gdrive and platform == "Colab":
    pm.mount_gdrive()

# --- Lightning AI specific environment tweaks ---
if platform == "LightningAI":
    # conda piece (if conda exists in the image)
    conda_install("libstdcxx-ng=13")
    # pip pins / extras
    pip_install("numpy<2.0", check=False)
    pip_install("scikit-image", "gradio", "moviepy", "plotly", check=False)
    # If requirements.txt exists in CWD, install it
    if os.path.exists("requirements.txt"):
        pip_install("-r", "requirements.txt")

# --- Install PyTorch3D (handles platform differences & fallbacks) ---
installer = install_tools.PyTorch3DInstaller(platform, local_path)
installer.install()

# --- Extra libraries (quiet-ish) ---
# Original line had: trimesh pyrender opencv-python matplotlib pytorch-lightning
pip_install("trimesh", "pyrender", "opencv-python", "matplotlib", "pytorch-lightning", check=False)

# --- Download plot_image_grid.py if missing ---
filename = "plot_image_grid.py"
url = "https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py"
if not os.path.exists(filename):
    print(f"Downloading {filename} ...")
    try:
        urllib.request.urlretrieve(url, filename)
        print("Saved to", pathlib.Path(filename).resolve())
    except Exception as e:
        print("Download failed:", e)

# --- gdown ---
pip_install("gdown", extra=["--quiet"], check=False)
print("✅ Setup complete.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.4 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'installation_tools'

time: 8.51 ms (started: 2025-10-01 22:09:15 +00:00)
